In [1]:
import os 
os.chdir("../")
%pwd

'c:\\Users\\hp\\Files\\DataScience\\End-To-End Projects\\Chicken-Disease-Classification-Projects'

In [2]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path 
    trained_model_path:Path 
    base_model_updated:Path 
    training_data: Path
    params_epochs: int 
    params_batch_size: int 
    params_is_augmentation: bool 
    params_image_size: list
    params_learning_rate: float


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.utils.common import read_yaml, create_directories 
from cnnClassifier.constants import * 

class ConfigurationManager:
    def __init__(self, 
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks 
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log)
        ])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=config.tensorboard_root_log,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model 
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images") 
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir= Path(training.root_dir), 
            trained_model_path = Path(training.trained_model_path), 
            base_model_updated = Path(prepare_base_model.updated_base_model_path), 
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE, 
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE,
            params_learning_rate = params.LEARNING_RATE
        )
        return training_config

In [4]:
import time 
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from cnnClassifier import logging

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [ ]:
import tensorflow as tf
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def get_base_model(self):
        logging.info("getting base model")
        self.model = tf.keras.models.load_model(
            self.config.base_model_updated
        )
        # Recompile the model with a new optimizer instance after loading
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        logging.info("train valid generator")

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split= 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= "validation",
            shuffle= True,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="training",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod 
    def save_model(path, model):
        model.save(path)

    def train(self, callback_list: list):


        
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [6]:
try:
    config = ConfigurationManager() 
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config= prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
     
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model() 
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e

[2025-10-05 17:28:45,537] [INFO] [root:read_yaml:22] - reading yaml file -> config\config.yaml successed.
[2025-10-05 17:28:45,545] [INFO] [root:read_yaml:22] - reading yaml file -> params.yaml successed.
[2025-10-05 17:28:45,548] [INFO] [root:create_directories:42] - folder 'artifacts' already exist.
[2025-10-05 17:28:45,551] [INFO] [root:create_directories:42] - folder 'artifacts\prepare_callbacks\checkpoint' already exist.
[2025-10-05 17:28:45,554] [INFO] [root:create_directories:42] - folder 'artifacts\prepare_callbacks\tensorboard_log' already exist.
[2025-10-05 17:28:45,557] [INFO] [root:create_directories:42] - folder 'artifacts\training' already exist.
[2025-10-05 17:28:45,545] [INFO] [root:read_yaml:22] - reading yaml file -> params.yaml successed.
[2025-10-05 17:28:45,548] [INFO] [root:create_directories:42] - folder 'artifacts' already exist.
[2025-10-05 17:28:45,551] [INFO] [root:create_directories:42] - folder 'artifacts\prepare_callbacks\checkpoint' already exist.
[2025-1

c:\Users\hp\Files\DataScience\Anaconda\envs\cnn-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\hp\Files\DataScience\Anaconda\envs\cnn-env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
c:\Users\hp\Files\DataScience\Anaconda\envs\cnn-env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4937 - loss: 12.1697

c:\Users\hp\Files\DataScience\Anaconda\envs\cnn-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-10-05 17:31:44,773] [WARNING] [absl:save_model:83] - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
19/19 ━━━━━━━━━━━━━━━━━━━━ 175s 9s/step - accuracy: 0.5541 - loss: 11.7248 - val_accuracy: 0.5156 - val_loss: 15.0378
19/19 ━━━━━━━━━━━━━━━━━━━━ 175s 9s/step - accuracy: 0.5541 - loss: 11.7248 - val_accuracy: 0.5156 - val_loss: 15.0378
[2025-10-05 17:31:45,032] [WARNING] [absl:save_model:83] - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
[2025-10-05 17:31:45,032] [WARNING] [absl:save_model:83] - You are saving your model